In [156]:
import pandas as pd

chatbot_data = pd.read_csv("chatbot_data.csv")

chatbot_data_100 = chatbot_data.head(50)
r, c = chatbot_data_100.shape
# print(chatbot_data.loc[2][0])

# for i in range(0,r):
#     print(chatbot_data_100.loc[i][3])

In [192]:
Categories = []
for i in range(0,len(chatbot_data)):
    if chatbot_data.iloc[i][3] not in Categories and pd.isnull(chatbot_data.iloc[i][3]) is False:
        Categories.append(chatbot_data.loc[i][3])
    

In [193]:
Categories

['MSISE',
 'F1 student',
 'Admissions',
 'GAPE',
 'MISC',
 'MSSE',
 'CMPE',
 'WST',
 'Major',
 'OU',
 'Graduation',
 'Financial Aid',
 'MSCS',
 'admissions',
 'ISSS',
 'Library',
 'financial Aid',
 'graduation',
 'Health Center',
 'Sports',
 'Canvas',
 'Zoom']

In [194]:
ps = PorterStemmer()

In [195]:
import spacy
import nltk
from nltk.stem import PorterStemmer
import warnings; 
warnings.simplefilter('ignore')

nlp = spacy.load('en')

synonym_dict = {}
synonym_dict[('United States', 'USA', 'America', 'United States of America',)] = 'US'
synonym_dict[('International student', 'international students')] = 'F1 student'
synonym_dict[('San Jose State University', 'SJ State University')] = 'SJSU'
synonym_dict[('International cell', 'International student and scholar services ')] = 'ISSS'
synonym_dict[('Graduate admission and programs evaluation', 'Graduate admissions and program evaluation',)] = 'GAPE'
synonym_dict[('Masters of Science in Software Engineering', 'MS in Software Engineering')] = 'MSSE'
synonym_dict[('MS-ISE', 'MS ISE', 'Masters in Industrial and Systems Engineering', 'MS in Industrial and Systems Engineering', 'Masters in ISE', 'MS in ISE')] = 'MSISE'
synonym_dict[('Computer Engineering', 'Computers Engineering')] = 'CMPE'
synonym_dict[('Software Engineering', 'SW Engineering')] = 'SE'
synonym_dict[('General Education', 'General Education')] = 'GE'
synonym_dict[('Writing skills test', 'Writing skill test')] = 'WST'
synonym_dict[('lower division', 'lower-division')] = 'LD'
synonym_dict[('California state university', 'California state university')] = 'CSU'
synonym_dict[('Former Student Returning', 'Former Student Returning')] = 'FSR'
synonym_dict[('Grade point average', 'Grade point average')] = 'GPA'
synonym_dict[('Open University', 'Open University')] = 'OU'
synonym_dict[('full-time', 'fulltime')] = 'full time'
synonym_dict[('Free Application for Federal Student Aid', 'Free Application for Federal Students Aid')] = 'FAFSA'
synonym_dict[('MS in Computer Science', 'Masters in CS', 'Masters in Computer Science', 'MS in CS')] = 'MSCS'
synonym_dict[('part-time', 'parttime')] = 'part time'
synonym_dict[('credit/no-credit', 'credit no-credit')] = 'CR/NCR'

stopwords = ["is", "an", "the", "are", "am", "was", "were", "I", "we", "will", "would", "a"]
def normalize_string(string):
    test_string = ""
    for word in string:
        test_string += word.text.lower()
        test_string += " "
    
    
    for key in synonym_dict:
        for t in key:
            strng = t.lower()
            if strng in test_string:
                test_string = test_string.replace(strng, synonym_dict[key])
                
    return nlp(test_string)


def preprocess(sentence_1):
    s1_stem = ""
    
    for word in sentence_1:
        if word.text in stopwords:
            continue
        
        rootWord = ps.stem(str(word.text))
        s1_stem += rootWord
        s1_stem += " "
    
    return s1_stem

    
def find_similarity(sentence_1, sentence_2):
    
    similarity_score = nlp(sentence_1).similarity(nlp(sentence_2))
    return similarity_score

Bot_data = []
for i in range(0,len(chatbot_data)):
    database_question = chatbot_data.loc[i][0]
    database_question = nlp(database_question)
    database_question = normalize_string((database_question))
    database_question = preprocess((database_question))
    Bot_data.append(database_question)
    
context = ""



In [217]:
# incoming_question = "I am not yet disqualiefied from San Jose State University, do I still need to file for a reinstatement petition"
incoming_question = "What is the application deadline?"
print("Incoming question : \n", incoming_question)
flag = 0
incoming_question = preprocess(normalize_string(nlp(incoming_question)))

m = 0.0
index = 0
for i in range(0,len(Bot_data)):
    
    sim = find_similarity(incoming_question, Bot_data[i])
    if (sim > m):
        m = sim
        index = i;

print("")
print("Answer : \n",chatbot_data.loc[index][1])
if pd.isnull(chatbot_data.loc[index][2]) is False:
    print(" ",chatbot_data.loc[index][2])

Incoming question : 
 What is the application deadline

Answer : 
 University admission deadlines are listed at
  http://www.sjsu.edu/gape/prospective_students/deadlines/


In [218]:
incoming_question = "What is Open University?"
print("Incoming question : \n", incoming_question)
flag = 0
incoming_question = preprocess(normalize_string(nlp(incoming_question)))

for word in nlp(incoming_question):
    if word.text in Categories:
        context = word.text
        flag = 1
        break
    
m = 0.0
index = 0
for i in range(0,len(Bot_data)):
    
    sim = find_similarity(incoming_question, Bot_data[i])
    if (sim > m):
        m = sim
        index = i;

print("")
print("Answer : \n",chatbot_data.loc[index][1])
if pd.isnull(chatbot_data.loc[index][2]) is False:
    print(" ",chatbot_data.loc[index][2])

Incoming question : 
 What is Open University?

Answer : 
  "Open University," or "Continuing Education" at SJSU allows individuals that are not registered in a degree program to take university courses. There are a number of restrictions.
                     


In [220]:
incoming_question = "What is the application deadline?"
print("Incoming question : \n", incoming_question)
flag = 0
incoming_question = preprocess(normalize_string(nlp(incoming_question)))

for word in nlp(incoming_question):
    if word.text in Categories:
        context = word.text
        flag = 1
        break

if flag == 0:
    incoming_question += context


m = 0.0
index = 0
for i in range(0,len(Bot_data)):
    
    sim = find_similarity(incoming_question, Bot_data[i] + chatbot_data.loc[i][3])
    if (sim > m):
        m = sim
        index = i;

print("")
print("Answer : \n",chatbot_data.loc[index][1])
if pd.isnull(chatbot_data.loc[index][2]) is False:
    print(" ",chatbot_data.loc[index][2])

Incoming question : 
 What is the application deadline?

Answer : 
  Please see registration website for Open University: https://www.sjsu.edu/openuniversity/registration/                     
